### Versi 2

In [1]:
# ===========================================
# Cell 1. Import Library dan Setup Environment
# ===========================================

import os
import json
import random
from pathlib import Path
from collections import Counter

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler
import zipfile
import io

# Gunakan GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device yang digunakan:", device)

Device yang digunakan: cuda


In [2]:
# ===========================================
# Cell 2. Fungsi Bantuan Umum
# ===========================================

from matplotlib.colors import ListedColormap

def seed_everything(seed=42):
    """Menetapkan seed random agar hasil eksperimen bisa direplikasi"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything(42)

def visualize_tile(x_tile, y_true=None, y_pred=None, json_path=None,class_names=None, idx=0):
    """
    Menampilkan citra tile beserta mask ground-truth dan prediksi
    """
    if isinstance(x_tile, torch.Tensor):
        x = x_tile.cpu().numpy()
        x = np.transpose(x, (1,2,0))  # ubah dari [B,H,W] -> [H,W,B]
    else:
        x = x_tile

    # menampilkan pseudo-RGB (karena data hyperspectral)
    B = x.shape[2]
    b1, b2, b3 = int(B*0.05), int(B*0.5), int(B*0.9)
    rgb = x[..., [b1, b2, b3]]
    rgb_norm = (rgb - rgb.min()) / (rgb.max() - rgb.min() + 1e-9)

    # Coba baca colormap dari file JSON
    if json_path and os.path.exists(json_path):
        with open(json_path, "r") as f:
            label_info = json.load(f)
        custom_colors = [c["color"][:7] for c in label_info]
        cmap = ListedColormap(custom_colors)
    else:
        print("File json tidak terbaca, menggunakan cmap tab20")
        cmap = "tab20"  # fallback

        

    # Visualisasi
    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1); plt.imshow(rgb_norm); plt.title("Citra (Pseudo-RGB)")
    if y_true is not None:
        plt.subplot(1,3,2); plt.imshow(y_true, cmap=cmap); plt.title("Ground Truth")
    if y_pred is not None:
        plt.subplot(1,3,3); plt.imshow(y_pred, cmap=cmap); plt.title("Prediksi")
    plt.show()



In [3]:
# =========================================================
# Cell 3. Dataset Loader (SeaweedDataset) dan Label Mapping
# =========================================================

def load_label_mapping(json_path):
    """Membaca file label_classes.json untuk mapping id ke nama kelas"""
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    idx_to_name = {i: item["name"] for i, item in enumerate(data)}
    return idx_to_name

def normalize_reflectance(cube):
    """Menormalkan nilai reflektansi ke rentang 0-1 per tile"""
    cube = np.nan_to_num(cube).astype(np.float32)
    min_val = np.nanmin(cube)
    max_val = np.nanmax(cube)
    if max_val > min_val:
        cube = (cube - min_val) / (max_val - min_val)
    return cube


class SeaweedDataset(Dataset):
    """
    Dataset hemat memori berbasis file .npy hasil konversi.
    Membaca tile langsung dari disk menggunakan mmap_mode="r".
    """
    def __init__(self, data_files, label_map, tile_size=128, normalize=True):
        self.data_files = data_files
        self.label_map = label_map
        self.tile_size = tile_size
        self.normalize = normalize

        # Daftar pasangan (file_x, file_y)
        self.pairs = []
        for f in data_files:
            if f.endswith("_x.npy"):
                fy = f.replace("_x.npy", "_y.npy")
                if os.path.exists(fy):
                    self.pairs.append((f, fy))
        
        # Hanya menyimpan indeks tile berdasarkan ukuran file .npy
        self.index = []  
        for file_idx, (fx, fy) in enumerate(self.pairs):
            x = np.load(fx, mmap_mode="r")
            H, W, _ = x.shape
            for i in range(0, H - tile_size + 1, tile_size):
                for j in range(0, W - tile_size + 1, tile_size):
                    self.index.append((file_idx, i, j))
            del x  # bebaskan referensi memori

        print(f"[INFO] Total tile terdaftar: {len(self.index)} dari {len(self.pairs)} file")

    def __len__(self):
        return len(self.index)

    def __getitem__(self, idx):
        file_idx, i, j = self.index[idx]
        fx, fy = self.pairs[file_idx]
        
        # Memuat tile menggunakan mmap
        x = np.load(fx, mmap_mode="r")[i:i+self.tile_size, j:j+self.tile_size, :]
        y = np.load(fy, mmap_mode="r")[i:i+self.tile_size, j:j+self.tile_size]

        # Abaikan tile kosong (semua 0)
        if not np.any(y > 0):
            # Jika tile kosong, ambil tile lain secara acak agar batch tetap penuh
            return self.__getitem__(np.random.randint(0, len(self.index)))

        if self.normalize:
            x = normalize_reflectance(x)

        # Konversi ke tensor
        x_tensor = torch.tensor(x.transpose(2, 0, 1), dtype=torch.float32)
        y_tensor = torch.tensor(y, dtype=torch.long)
        return x_tensor, y_tensor

def detect_actual_classes(pairs):
    """Scan semua file y.npy untuk mendeteksi kelas yang benar-benar ada"""
    found = set()
    for _, fy in pairs:
        y = np.load(fy, mmap_mode="r")
        found |= set(np.unique(y))
    found = sorted(list(found))
    print(f"[INFO] Kelas AKTUAL yang ditemukan di dataset: {found}")
    return found

In [4]:
# ==============================================
# Cell 4. Load Dataset dan FILE-LEVEL Splitting
# ==============================================

data_dir = "../data/npy_converted"
label_json_path = "../data/annotation/segmentation_masks/label_classes.json"

label_map = load_label_mapping(label_json_path)
print(f"Jumlah total kelas di JSON: {len(label_map)}")

# Ambil semua file _x.npy
all_x_files = sorted([os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith("_x.npy")])
pairs = [(fx, fx.replace("_x.npy", "_y.npy")) for fx in all_x_files if os.path.exists(fx.replace("_x.npy", "_y.npy"))]

print(f"Total pasangan file X-Y ditemukan: {len(pairs)} (expected: 18)")
for p in pairs[:5]:
    print("Contoh:", os.path.basename(p[0]), "<->", os.path.basename(p[1]))

# Split deterministik berbasis urutan nama (11 train, 5 val, 3 test)
train_pairs = pairs[:11]
val_pairs   = pairs[11:16]
test_pairs  = pairs[16:]

print("\n=== FINAL SPLIT PER FILE ===")
print(f"Train : {len(train_pairs)}")
print(f"Val   : {len(val_pairs)}")
print(f"Test  : {len(test_pairs)}")

# DETEKSI kelas aktual yang benar-benar muncul (bukan ambil dari JSON)
actual_classes = detect_actual_classes(train_pairs + val_pairs + test_pairs)
num_classes_actual = len(actual_classes)
print(f"\nnum_classes SEMENTARA = {num_classes_actual} (nanti kita pakai ini untuk model)")

# Buat dataset per split
train_dataset = SeaweedDataset([p[0] for p in train_pairs], label_map, tile_size=128)
val_dataset   = SeaweedDataset([p[0] for p in val_pairs], label_map, tile_size=128)
test_dataset  = SeaweedDataset([p[0] for p in test_pairs], label_map, tile_size=128, normalize=False)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=4, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=4, shuffle=False)

print(f"\nTotal TILE train: {len(train_dataset)}, val: {len(val_dataset)}, test: {len(test_dataset)}")


Jumlah total kelas di JSON: 41
Total pasangan file X-Y ditemukan: 18 (expected: 18)
Contoh: massimal_smola_maholmen_202306211129-2_hsi_003_processed_x.npy <-> massimal_smola_maholmen_202306211129-2_hsi_003_processed_y.npy
Contoh: massimal_smola_maholmen_202306211129-2_hsi_004_processed_x.npy <-> massimal_smola_maholmen_202306211129-2_hsi_004_processed_y.npy
Contoh: massimal_smola_maholmen_202306211129-2_hsi_008_processed_x.npy <-> massimal_smola_maholmen_202306211129-2_hsi_008_processed_y.npy
Contoh: massimal_smola_maholmen_202306211129-2_hsi_009_processed_x.npy <-> massimal_smola_maholmen_202306211129-2_hsi_009_processed_y.npy
Contoh: massimal_smola_maholmen_202306211129-2_hsi_013_processed_x.npy <-> massimal_smola_maholmen_202306211129-2_hsi_013_processed_y.npy

=== FINAL SPLIT PER FILE ===
Train : 11
Val   : 5
Test  : 2
[INFO] Kelas AKTUAL yang ditemukan di dataset: [np.int32(0), np.int32(8), np.int32(12), np.int32(13), np.int32(14), np.int32(18), np.int32(38)]

num_classes SEMENTAR

In [5]:
# === CELL DEBUG (BUKAN CELL URUTAN ===

print(np.load(train_pairs[0][0], mmap_mode="r").shape)
print(np.load(train_pairs[1][0], mmap_mode="r").shape)
print(np.load(train_pairs[-1][0], mmap_mode="r").shape)

(2000, 900, 300)
(2000, 900, 300)
(2000, 900, 300)


In [ ]:
# ======================================================
# Cell 5. Model Fully Convolutional HybridSN (3D+2D CNN)
# ======================================================

class FCHybridSN(nn.Module):
    def __init__(self, in_bands=300, num_classes=7): # nilai num_classes dijadikan default 7
        super().__init__()
        self.conv3d_1 = nn.Conv3d(1, 16, (7,3,3), padding=(0,1,1))
        self.bn3d_1 = nn.BatchNorm3d(16)
        self.conv3d_2 = nn.Conv3d(16, 32, (5,3,3), padding=(0,1,1))
        self.bn3d_2 = nn.BatchNorm3d(32)
        self.conv3d_3 = nn.Conv3d(32, 64, (3,3,3), padding=(0,1,1))
        self.bn3d_3 = nn.BatchNorm3d(64)

        self._out_spec = in_bands - 12
        mid_ch = 256
        self.conv2d_1 = nn.Conv2d(64 * max(1, self._out_spec), mid_ch, 3, padding=1)
        self.bn2d_1 = nn.BatchNorm2d(mid_ch)
        self.conv2d_2 = nn.Conv2d(mid_ch, 128, 3, padding=1)
        self.bn2d_2 = nn.BatchNorm2d(128)
        self.conv2d_3 = nn.Conv2d(128, 64, 3, padding=1)
        self.bn2d_3 = nn.BatchNorm2d(64)
        self.classifier = nn.Conv2d(64, num_classes, 1)

    def forward(self, x):
        B, Bands, H, W = x.shape
        x3 = x.unsqueeze(1)
        x3 = F.relu(self.bn3d_1(self.conv3d_1(x3)))
        x3 = F.relu(self.bn3d_2(self.conv3d_2(x3)))
        x3 = F.relu(self.bn3d_3(self.conv3d_3(x3)))
        B, C3, out_spec, H, W = x3.shape
        x2 = x3.view(B, C3 * out_spec, H, W)
        x2 = F.relu(self.bn2d_1(self.conv2d_1(x2)))
        x2 = F.relu(self.bn2d_2(self.conv2d_2(x2)))
        x2 = F.relu(self.bn2d_3(self.conv2d_3(x2)))
        return self.classifier(x2)

# Ambil jumlah band langsung dari data TRAIN pertama
sample_x = np.load(train_pairs[0][0], mmap_mode="r")
in_bands_actual = sample_x.shape[2]  # ambil jumlah band asli dari npy

print(f"Band input aktual terdeteksi: {in_bands_actual}")

# GUNAKAN jumlah kelas AKTUAL (bukan 41 dari JSON!)
model = FCHybridSN(in_bands=in_bands_actual, num_classes=num_classes_actual).to(device)

print(model)
